In [1]:
import pandas as pd
import math
import numpy as np
from sklearn.cross_validation import train_test_split # to split the dataset for training and testing

/home/garvit/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# getting the ratings data in pandas dataframe
ratings = pd.read_csv('../Yahoo_movies_multi-criteria/data_movies.txt', sep='\t',names=['user_id', 'criterion1', 'criterion2', 'criterion3', 'criterion4', 'overall', 'movie_id', 'num'])
train, test = sample_split(ratings)

print(train)
print(test)

       user_id  criterion1  criterion2  criterion3  criterion4  overall  \
0            1           6           6           8          12        8   
35           1          11          13          12          12       12   
6            1           9          13          11          13       11   
10           1          13          12          11           9       12   
23           1          11          12          12          12       12   
38           1           9          10           9          10        9   
44           1           3           3           3          12        4   
47           1           9          10          10          11        8   
15           1           9          12           6          12       10   
42           1           3           2           2           1        2   
22           1           9          11          12          11       10   
12           1           9          11          10           8       11   
11           1          1

In [34]:
ratings.user_id.drop_duplicates().count() #number of users

6078

In [4]:
# split the datasets into smaller datasets
def split_dataset(ratings):
        df_group = ratings.groupby(['user_id'])
        list_datasets = list() # list containing all the small datasets
        temp = pd.DataFrame()
        count=0
        flag=0
        for key, item in df_group:
            count+=1
            temp = temp.append(item)
            if count==1000:
                if(len(list_datasets)==5 and flag!=1):
                    flag=1
                    count = count - 78
                    continue
                list_datasets.append(temp)
                temp = pd.DataFrame()
                count=0
        return list_datasets

In [5]:
# Spliting a sample into groups userwise
def sample_split(dataFrame):
    df_group = dataFrame.groupby('user_id')
    train = pd.DataFrame()
    test = pd.DataFrame()
    
    for key, item in df_group:
        train, test = split_train_test(item, train, test)
    
    return train, test

In [6]:
# spliting sample in 70% training data and 30% testing data
def split_train_test(dataFrame, train, test):
    temp_train, temp_test = train_test_split(dataFrame, test_size = 0.3, random_state=1212)# in this our main data is split into train and test
    # the attribute test_size=0.3 splits the data into 70% and 30% ratio. train=70% and test=30%
    train = train.append(temp_train)
    test = test.append(temp_test)
    
    return train, test